# Predicting Wing Lift and Drag

In [1]:
# Add path to src/CARPy, in case notebook is running locally
import os, sys, warnings
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..\\..\\..\\src")))
warnings.simplefilter("ignore")  # <-- Suppress warnings

***
## Introduction

*As a pre-requisite to this notebook, it is recommended readers first familiarise themselves with the available material on modelling aerofoils in CARPy.*

With the selection of candidate aerofoils, the performance of a 3D lifting geometry can now be estimated with CARPy.
This mapping of 2D aerofoil sections to 3D wing geometry requires:

1. WingSection, an object that dimensionalises the chord length and thickness of non-dimensional aerofoil geometries.
2. WingSections, a dictionary-like object that permits the definition of wing dihedral, sweep, and twist.

Much like the aerofoil performance prediction methods, wing performance is captured through a variety of methods in:

3. WingSolution, an object capturing basic performance coefficients of the wing.

Inverse design methods are not supported at this time.

***
## 1) `WingSection` objects

Start with relevant imports

In [2]:
from carpy.aerodynamics.aerofoil import NewAerofoil
from carpy.aerodynamics.wing import WingSection

There are several angles or reference planes along which a wing can be sliced to produce sections or aerofoils.
CARPy wing sections are currently defined in a vertical plane parallel to the aircraft's buttock line:

In [3]:
# Instantiate geometry that will define the wing shape
naca0012 = NewAerofoil.from_method.NACA("0012")
naca2412 = NewAerofoil.from_method.NACA("2412")
naca2418 = NewAerofoil.from_method.NACA("2418")

# Defining the starboard wing section geometries
wbl_0012 = WingSection(naca0012)
wbl_2412 = WingSection(naca2412)
wbl_2418 = WingSection(naca2418)

WingSection objects are not designed for users to be able to interact with the attributes of directly, and therefore attributes should be treated as read-only.
The original aerofoil section can be recovered:

In [4]:
wbl_0012.aerofoil.show()

But it is recommended that properties of a `WingSection` are set via the `WingSections` dictionary-like object.

***
## 2) `WingSections` objects

`WingSections` objects can be thought of as a dictionary of aerofoils, created by the user, that define the 3D geometry of the wing.
Users familiar with dictionaries should be comfortable with the concept that a dictionary can have a key-value pair designated with the following syntax:
    
    >>> my_dictionary[some_key] = some_value
    
    >>> print(my_dictionary)
    {some_key: some_value}

The difference in implementation between a regular dictionary is three-fold:

1. Only integers and floats are valid keys, such that all keys lie on a one-dimensional line of real numbers.
2. The hypothetical value for a key "C" that lies between two other keys "A" and "B" is obtained by linearly interpolating between keys "A" and "B".
3. The hypothetical value for a key "D" must be extrapolated from a single neighbour "B", which actually just makes the value of "D" identical to the value at "B".

We'll demonstrate this more concretely with an example. Begin with the relevant import:

In [5]:
from carpy.aerodynamics.wing import WingSections

# Define the wing geometry at arbitrary buttock-line stations 0 and 10
conceptwing = WingSections(b=12)  # Assign a span, b = 12 metres
conceptwing[0] = wbl_2418
conceptwing[10] = wbl_2412

Notice that each wing section defining the 3D geometry is identifiable through its location in memory (beginning with `0x`):

In [6]:
print(f"The defined wing sections are:\n{conceptwing=}")

##### A quick detour:

To prove points 2 and 3, we'll show that users can access aerofoil geometry between two defined aerofoils (aerofoil C), and an aerofoil extrapolated from a single bounding aerofoil (aerofoil D is identical to aerofoil B):

In [7]:
# To align with the explanation above, use the following keys
section_dict = {"A":0, "C":5, "B":10, "D":14}

print("* Here's the memory locations of each wing section object successfully evaluated:\n")
for (key, value) in section_dict.items():
    print(f"{key}:{value} aerofoil --> {conceptwing[value]}")
else:
    print("")

print("* Here's proof that the average of two adjacent sections [0] and [10] is the same as just taking [5]:")
aerofoil_interp_manual = (0.5 * conceptwing[0].aerofoil + 0.5 * conceptwing[10].aerofoil)  # averaging geometry
aerofoil_interp_auto = conceptwing[5].aerofoil  # interpolating geometry automatically behind the scenes

aerofoil_interp_manual.show()
aerofoil_interp_auto.show()
print("")

print("* Here's proof that section[B] and section[D] are the same section (same memory location):\n")
print(f"-->\t{(conceptwing[section_dict['B']] is conceptwing[section_dict['D']])=}")

##### End of detour, back to the notebook lesson at hand

it is necessary to specify angles of dihedral, twist, and sweep for 3D wing geometries.
Wing taper can also be optionally adjusted.

In [8]:
import numpy as np

# Define another section of the wing at another arbitrary station (in proportion to the original station definitions)
conceptwing[14] = wbl_0012

# Adjust the SI chord lengths of the wing (changing taper ratio, optional)
conceptwing[:].chord = 2.0  # Use [:] slicing to define the chord length at all stations currently defined
conceptwing[14].chord = 0.8  # The station defined at position [14] should have a custom chord length

# The dihedral angle, leading edge sweep angle, and geometric station twist angles are now specified
conceptwing[:].dihedral = np.radians(2)  # Upward dihedral on the whole wing for roll stability
conceptwing[:].sweep = np.radians(0)  # Define leading edge sweep everywhere
conceptwing[10:].sweep = np.radians(2)  # Update leading edge sweep for any stations starting at and including 10
conceptwing[14].twist = np.radians(1)  # Increased wing tip incidence, a.k.a wash-in

It doesn't look like much right now, but that's it!
You've defined a wing of arbitrary span, but definite chord length, twist, sweep, dihedral, and profile.

##### <span style="color:red">IMPORTANT NOTE</span>:

At this point, a wing's 3D geometry is fully defined. We can access parameters such as the standard mean chord, or aspect ratio of the wing:

In [9]:
# This can take a few seconds to run!
attributes = ["AR", "MAC", "MGC"]
print("; ".join([f"{x} = {getattr(conceptwing, x)}" for x in attributes]))

The above calculations are slow to carry out because many sample points are taken along the span of the wing.
In anticipation of frequent re-access to these attributes, the properties are _cached_, meaning subsequent calls to these attributes use the result computed at an earlier date:

In [10]:
# This doesn't take as long to run
print("; ".join([f"{x} = {getattr(conceptwing, x)}" for x in attributes]))

That's because the results of the first computation are stored in a private dictionary of the class:

In [11]:
print(conceptwing.__dict__)

CARPy does not expect users to modify a wing object during an analysis.
It is expected that users create a new wing for each wing to be analysed.
Failure to create a new wing will likely result in the use of incorrect use of cached attributes that no longer reflect properties of the current wing.

If one really must use the same wing object, please make use of the `.cache_clear()` method. The library author hopes this can one day be removed:

In [12]:
# You can trust the results of a modified wing after calling this, but again its use is discouraged.
conceptwing.cache_clear()

print(conceptwing.__dict__)

***
## 3) `WingSolution` objects

It is possible to carry out a rudimentary analysis of aerofoil performance in CARPy, returing `AerofoilSolution` objects.
These can be obtained through:

In [13]:
from carpy.aerodynamics.wing import PrandtlLLT  # Prandtl's lifting line theory for unswept wings
from carpy.aerodynamics.wing import HorseshoeVortex  # Span-wise horseshoe vortex method

All derivative methods of `WingSolution` have similar instantiation structures.

#### `PrandtlLLT`

PrandtlLLT is limited in analysis to unswept wings, full details in the docstring.

In [14]:
# Apply PLLT to a symmetrical wing of 12 metre span
flightconditions = {"altitude": 0, "TAS": 40, "alpha": np.radians(10)}
soln0 = PrandtlLLT(wingsections=conceptwing, **flightconditions)

public_methods = [x for x in dir(soln0) if not x.startswith("_")]
print(f"{public_methods=}\n")

for method_name in public_methods:
    print(f"{method_name:>8} = {getattr(soln0, method_name)}")

#### `HorseshoeVortex`

Horseshoe Vortex methods applied to the same wing take longer, but permit the analysis of asymmetric wings, section-wise lift coefficients, and even centre of pressure.

##### <span style="color:red">IMPORTANT NOTE</span>:

Users are asked to note that currently, this method assumes the centre of pressure for each vortex element sweeps by the same amount the leading edge does. This requires a fix in future, but for now, defining a leading edge sweep of 0 actually returns the performance of a wing where the section-wise centre of pressure has a sweep of 0 besides the dihedral direction (i.e. leading edge sweep varies unexpectedly).

In [15]:
# Apply Horseshoe Vortex to a symmetrical wing of 12 metre span
soln1 = HorseshoeVortex(wingsections=conceptwing, **flightconditions)

public_methods = [x for x in dir(soln1) if not x.startswith("_")]
print(f"{public_methods=}\n")

for method_name in public_methods:
    print(f"{method_name:>8} = {getattr(soln1, method_name)}")

#### `MixedBLDrag`

CARPy provides means of estimating skin friction drag for a generic wing with surface quality specified by providing the `esg_roughness` (equivalent sand grain roughness).

The component of drag due to skin friction can be estimated from definitions of geometry and the relevant flight conditions:

In [18]:
from carpy.aerodynamics.wing import MixedBLDrag
from carpy.utility import Quantity, constants as co

# Make a drag prediction
drag_prediction = MixedBLDrag(
    wingsections=conceptwing,  # <-- Geometry
    altitude=0, TAS=Quantity(185, "kt"),  # <-- Flight conditions
    esg_roughness=float(co.MATERIAL.esg_roughness.mattepaint_careful)
)

In [19]:
print(f"A CDf of {drag_prediction.CDf} is predicted for the concept wing.")

***
## `*future*` objects

The plan is to eventually supercede `WingSolution` (valid for a very particular set of flight conditions) with an object that will conveniently interpolate lift, drag, centre of pressure, and other metrics of wing performance as a function of the wing's angle of attack.

For now, the burden is upon CARPy's users (sorry!).